In [1]:
import numpy as np
import matplotlib.pyplot as plt
from DynamicModel_Package.ModelBase import DynamicModel
import pandas as pd
np.random.seed(7)

In [2]:
def derivative_M (variables,parameters):
    M = variables['M']
    I = variables['I']
    a = parameters['a']
    c = parameters ['c']
    thresh = parameters['thresh']
    return (a*M)*int(M>thresh) - c*M*I

def derivative_I (variables,parameters):
    M = variables['M']
    I = variables['I']
    I_max = parameters['I_max']
    b = parameters['b']
    rI = parameters['rI']
    return b*M*(I_max-I)-rI*I

def derivative_D (variables,parameters):
    M = variables['M']
    D = variables['D']
    I = variables['I']
    d = parameters['d']
    g = parameters['g']
    rD = parameters['rD']
    return (d*M + g*I)- rD*D

def sample_parameters(num_samples=1000, a_range=(4.5, 14.5), b_range=(0.05,0.15), 
                      c_range=(10, 30),d_range=(0.1, 2),g_range=(0.1, 2),
                      rI_range=(0.05, 0.15), rD_range=(0.1, 2), 
                      I_max_range=(0.1, 10),
                      a_default = 10, b_default = 0.1, c_default = 15, d_default = 0.1, g_default = 2,
                      rI_default = 0.1,rD_default = 2, I_default = 5,
                      sample_a=False, sample_b=False, sample_c=False, sample_d=False, 
                      sample_g=False, sample_rI=False, sample_rD=False, sample_I_max=False,
                      sample_d_dist = 'uniform',
                      sample_rD_dist = 'uniform'):

    
    a_vals = np.random.uniform(*a_range, num_samples) if sample_a else np.ones(num_samples) * a_default
    b_vals = np.random.uniform(*b_range, num_samples) if sample_b else np.ones(num_samples) * b_default
    c_vals = np.random.uniform(*c_range, num_samples) if sample_c else np.ones(num_samples) * c_default
    if sample_d:
        if sample_d_dist=='uniform':
            d_vals = np.random.uniform(*d_range, num_samples)
        elif sample_d_dist=='halfnorm':
            from scipy.stats import halfnorm
            d_vals = halfnorm.rvs(loc=d_range[0],scale=d_range[1],size = num_samples)
        elif sample_d_dist=='gaussian':
            d_vals = np.random.normal(loc=d_range[0],scale=d_range[1],size = num_samples)
            d_vals[d_vals<0] = 0

    else:
        d_vals = np.ones(num_samples) * d_default
    g_vals = np.random.uniform(*g_range, num_samples) if sample_g else np.ones(num_samples) * g_default
    if sample_rD:
        if sample_rD_dist=='uniform':
            rD_vals = np.random.uniform(*rD_range, num_samples)
        elif sample_rD_dist=='halfnorm':
            from scipy.stats import halfnorm
            rD_vals = halfnorm.rvs(loc=rD_range[0],scale=rD_range[1],size = num_samples)
        elif sample_rD_dist=='gaussian':
            rD_vals = np.random.normal(loc=rD_range[0],scale=rD_range[1],size = num_samples)
            rD_vals[rD_vals<0] = 0
        elif sample_rD_dist=='loguniform':
            from scipy.stats import loguniform
            rD_vals = loguniform.rvs(rD_range[0],rD_range[1],size = num_samples)

    else:
        rD_vals = np.ones(num_samples) * rD_default
    rI_vals = np.random.uniform(*rI_range, num_samples) if sample_rI else np.ones(num_samples) * rI_default
    I_vals = np.random.uniform(*I_max_range, num_samples) if sample_I_max else np.ones(num_samples) * I_default
    
    return a_vals, b_vals,c_vals,d_vals, g_vals,rD_vals,rI_vals, I_vals


In [3]:
ranges = {'rD_range':(6,2),
                         'I_max_range':(4,12),
                         'a_range':(5,8),
                         'b_range':(1,7),
                         'd_range':(2,1),
                         'g_range':(1,5),
                         'c_range':(2,5)}
num_samples = 200
vals = sample_parameters(num_samples=num_samples,
                         sample_a=True,
                         sample_b=True,
                         sample_c= True,
                         sample_d = True,
                         sample_g=False,
                         sample_I_max=True,
                         sample_rD=True,
                         sample_rI=False,
                         rD_range=ranges['rD_range'],
                         I_max_range = ranges['I_max_range'],
                         b_range = ranges['b_range'],
                         g_range=ranges['g_range'],
                         c_range = ranges['c_range'],
                         d_range = ranges['d_range'],
                         a_range = ranges['a_range'],
                         sample_d_dist='gaussian',
                         sample_rD_dist='gaussian')      
df_microbial = pd.DataFrame(columns = ['a','b','c','d','g','rD','rI','I_max','E','I','M','D'])
df_immune = pd.DataFrame(columns = ['a','b','c','d','g','rD','rI','I_max','E','I','M','D'])
np.random.seed(7)
for i in range(num_samples):
    MDI_model = DynamicModel()
    parameters= {}
    parameters['a'] = vals[0][i]
    parameters['b'] = vals[1][i]
    parameters['c'] = vals[2][i]
    parameters['d'] = vals[3][i]
    parameters['g'] = 0
    parameters['rD'] = vals[5][i]
    parameters['rI'] = vals[6][i]
    parameters['I_max'] = vals[7][i]
    parameters['thresh'] = 0.1  
    MDI_model.add_variable('M',derivative_M,parameters)
    MDI_model.add_variable('D',derivative_D,parameters)
    MDI_model.add_variable('I',derivative_I,parameters)
    intg = MDI_model.euler_integrate_keep_positive({'M':1,'I':0.01,'D':0,'t':0},t_final = 5, dt = 0.025)
    row = {'a':[vals[0][i]],'b':[vals[1][i]],'c':[vals[2][i]],'d':[vals[3][i]],'g':[0],'rD':[vals[5][i]],'rI':[vals[6][i]],'I_max':[vals[-1][i]],
           'I':[intg['I'].max()],'M':[intg['M'].max()],'D':[intg['D'].max()],'E':[intg['I'].max()+intg['D'].max()+4*vals[5][i]+vals[6][i]]}
    df_microbial = pd.concat([df_microbial,pd.DataFrame(row)],ignore_index=True)

for i in range(num_samples):
    MDI_model = DynamicModel()
    parameters= {}
    parameters['a'] = vals[0][i]
    parameters['b'] = vals[1][i]
    parameters['c'] = vals[2][i]
    parameters['d'] = 0
    parameters['g'] = vals[3][i]
    parameters['rD'] = vals[5][i]
    parameters['rI'] = vals[6][i]
    parameters['I_max'] = vals[7][i]
    parameters['thresh'] = 0.1  
    MDI_model.add_variable('M',derivative_M,parameters)
    MDI_model.add_variable('D',derivative_D,parameters)
    MDI_model.add_variable('I',derivative_I,parameters)
    intg = MDI_model.euler_integrate_keep_positive({'M':1,'I':0.01,'D':0,'t':0},t_final = 5, dt = 0.025)
    row = {'a':[vals[0][i]],'b':[vals[1][i]],'c':[vals[2][i]],'d':[0],'g':[vals[3][i]],'rD':[vals[5][i]],'rI':[vals[6][i]],'I_max':[vals[-1][i]],
           'I':[intg['I'].max()],'M':[intg['M'].max()],'D':[intg['D'].max()],'E':[intg['I'].max()+intg['D'].max()+4*vals[5][i]+vals[6][i]]}
    df_immune = pd.concat([df_immune,pd.DataFrame(row)],ignore_index=True)

columns_for_zscore = ['D','M','I']
for column in columns_for_zscore:
    new_column_name = column+'.zscore'
    df_immune[new_column_name] = (df_immune[column]-df_immune[column].mean())/(df_immune[column].std())
    df_microbial[new_column_name] = (df_microbial[column]-df_microbial[column].mean())/(df_microbial[column].std())

remove_outliers = True
use_zscore = True
if remove_outliers:
    if use_zscore:
        df_microbial = df_microbial[abs(df_microbial['M.zscore'])<=3]
        df_microbial = df_microbial[abs(df_microbial['D.zscore'])<=3]
        df_immune = df_immune[abs(df_immune['M.zscore'])<=3]
        df_immune = df_immune[abs(df_immune['D.zscore'])<=3]

    else:
        df_microbial = df_microbial[abs(df_microbial['M.zscore'])<=5]
        df_microbial = df_microbial[abs(df_microbial['D.zscore'])<=40]
        df_immune = df_immune[abs(df_immune['M.zscore'])<=5]
        df_immune = df_immune[abs(df_immune['D.zscore'])<=40]


In [4]:
df_microbial.to_csv('df_microbial.csv')
df_immune.to_csv('df_immune.csv')

In [ ]:
ranges = {'rD_range':(1,0.2),
                         'I_max_range':(4,12),
                         'a_range':(5,8),
                         'b_range':(1,7),
                         'd_range':(1,4),
                         'g_range':(0.1,0.1),
                         'c_range':(2,5)}

np.random.seed(7)
num_samples = 200
vals = sample_parameters(num_samples=num_samples,
                         sample_a=True,
                         sample_b=True,
                         sample_c= True,
                         sample_d = True,
                         sample_g=True,
                         sample_I_max=True,
                         sample_rD=True,
                         sample_rI=False,
                         rD_range=ranges['rD_range'],
                         I_max_range = ranges['I_max_range'],
                         b_range = ranges['b_range'],
                         g_range=ranges['g_range'],
                         c_range = ranges['c_range'],
                         d_range = ranges['d_range'],
                         a_range = ranges['a_range'],
                         sample_rD_dist='gaussian',
                         sample_d_dist='uniform',)      
df_rbc = pd.DataFrame(columns = ['a','b','c','d','g','rD','rI','I_max','E','I','M','D'])
for i in range(num_samples):
    MDI_model = DynamicModel()
    parameters= {}
    parameters['a'] = vals[0][i]
    parameters['b'] = vals[1][i]
    parameters['c'] = vals[2][i]
    parameters['d'] = vals[3][i]
    parameters['g'] = vals[4][i]
    parameters['rD'] = vals[5][i]
    parameters['rI'] = vals[6][i]
    parameters['I_max'] = vals[7][i]
    parameters['thresh'] = 0.1  
    MDI_model.add_variable('M',derivative_M,parameters)
    MDI_model.add_variable('D',derivative_D,parameters)
    MDI_model.add_variable('I',derivative_I,parameters)
    intg = MDI_model.euler_integrate_keep_positive({'M':1,'I':0.01,'D':0,'t':0},t_final = 5, dt = 0.025)
    row = {'a':[vals[0][i]],'b':[vals[1][i]],'c':[vals[2][i]],'d':[vals[3][i]],'g':[vals[4][i]],'rD':[vals[5][i]],'rI':[vals[6][i]],'I_max':[vals[-1][i]],
           'I':[intg['I'].max()],'M':[intg['M'].max()],'D':[intg['D'].max()],'E':[2*intg['I'].max()+intg['D'].max()+4*vals[5][i]+vals[6][i]]}
    df_rbc = pd.concat([df_rbc,pd.DataFrame(row)],ignore_index=True)

columns_for_zscore = ['D','M','I']
for column in columns_for_zscore:
    new_column_name = column+'.zscore'
    df_rbc[new_column_name] = (df_rbc[column]-df_rbc[column].mean())/(df_rbc[column].std())
remove_outliers=True
remove_outliers_zscore=True
recalc_zscore = False
if remove_outliers:
    if remove_outliers_zscore:
        df_rbc = df_rbc[abs(df_rbc['M.zscore'])<=3]
        df_rbc = df_rbc[abs(df_rbc['D.zscore'])<=3]
    else:
        df_rbc = df_rbc[abs(df_rbc['M'])<=3]
        df_rbc = df_rbc[abs(df_rbc['D'])<=4]

    columns_for_zscore = ['D','M','I']
    if recalc_zscore:
        for column in columns_for_zscore:
            new_column_name = column+'.zscore'
            df_rbc[new_column_name] = (df_rbc[column]-df_rbc[column].mean())/(df_rbc[column].std())
df_rbc.to_csv('df_rbc.csv')

In [ ]:
df_rbc = df_rbc[abs(df_rbc['M'])<=5]
df_rbc = df_rbc[abs(df_rbc['D'])<=10]
columns_for_zscore = ['D','M','I']
for column in columns_for_zscore:
    new_column_name = column+'.zscore'
    df_rbc[new_column_name] = (df_rbc[column]-df_rbc[column].mean())/(df_rbc[column].std())
